In [9]:
!pip install pyspark
!pip install numpy
!pip install setuptools

  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)


In [10]:
import pyspark

In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName("MLLIB").getOrCreate()

In [13]:
df = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [14]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [15]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [19]:
# Predicting salary
#  independent feature : [Age, Experience]
from pyspark.ml.feature import VectorAssembler

In [22]:
featureAssembler=VectorAssembler(inputCols=['age','Experience'],outputCol='indep_features')

In [23]:
training = df
output=featureAssembler.transform(training)

In [24]:
output.show()

+---------+---+----------+------+--------------+
|     Name|age|Experience|Salary|indep_features|
+---------+---+----------+------+--------------+
|    Krish| 31|        10| 30000|   [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|    [30.0,8.0]|
|    Sunny| 29|         4| 20000|    [29.0,4.0]|
|     Paul| 24|         3| 20000|    [24.0,3.0]|
|   Harsha| 21|         1| 15000|    [21.0,1.0]|
|  Shubham| 23|         2| 18000|    [23.0,2.0]|
+---------+---+----------+------+--------------+



In [26]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'indep_features']

In [27]:
finalized_data = output.select('indep_features','Salary')

In [28]:
finalized_data.show()

+--------------+------+
|indep_features|Salary|
+--------------+------+
|   [31.0,10.0]| 30000|
|    [30.0,8.0]| 25000|
|    [29.0,4.0]| 20000|
|    [24.0,3.0]| 20000|
|    [21.0,1.0]| 15000|
|    [23.0,2.0]| 18000|
+--------------+------+



In [29]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regression = LinearRegression(featuresCol='indep_features', labelCol='Salary')
regression = regression.fit(train_data)

24/04/03 13:06:45 WARN Instrumentation: [c4d3044a] regParam is zero, which might cause numerical instability and overfitting.
24/04/03 13:06:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/03 13:06:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/04/03 13:06:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [30]:
regression.coefficients

DenseVector([-64.8464, 1584.7554])

In [31]:
regression.intercept

15414.10693970376

In [32]:
pred_results = regression.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------+------+-----------------+
|indep_features|Salary|       prediction|
+--------------+------+-----------------+
|    [24.0,3.0]| 20000|18612.05915813422|
+--------------+------+-----------------+

